In [24]:
import torch
from torch import nn
from torch import optim

In [25]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]])
target = torch.tensor([[0],[0],[1],[1.]])

# A Toy Model
model = nn.Linear(2,1)

In [26]:
def cenTrain():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)

In [27]:
cenTrain()

tensor(3.5069)
tensor(1.5959)
tensor(0.9790)
tensor(0.6284)
tensor(0.4061)
tensor(0.2631)
tensor(0.1708)
tensor(0.1112)
tensor(0.0726)
tensor(0.0476)
tensor(0.0313)
tensor(0.0207)
tensor(0.0137)
tensor(0.0092)
tensor(0.0062)
tensor(0.0042)
tensor(0.0028)
tensor(0.0019)
tensor(0.0013)
tensor(0.0009)


Above was an centralized model training example, let's see federated way now.

In [28]:
import syft as sy
hook = sy.TorchHook(torch)

In [29]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [30]:
# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)

target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

In [31]:
from syft.federated.floptimizer import Optims
workers = ['bob', 'alice']
optims = Optims(workers, optim=optim.Adam(params=model.parameters(),lr=0.1))

In [33]:
def fedTrain():
    # Training Logic
    for iter in range(10):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            model.send(data.location)
            
            #Call the optimizer for the worker using get_optim
            opt = optims.get_optim(data.location.id)
            #print(data.location.id)

            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)

            # 3) calculate how much we missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # 5) change those weights
            opt.step()
            
            # NEW) get model (with gradients)
            model.get()

            # 6) print our progress
            print(loss.get().data) # NEW) slight edit... need to call .get() on loss\
    
# federated averaging

In [36]:
fedTrain()

tensor(0.0013)
tensor(0.0035)
tensor(0.0004)
tensor(0.0080)
tensor(0.0004)
tensor(0.0201)
tensor(0.0016)
tensor(0.0369)
tensor(0.0039)
tensor(0.0538)
tensor(0.0064)
tensor(0.0658)
tensor(0.0086)
tensor(0.0696)
tensor(0.0098)
tensor(0.0647)
tensor(0.0100)
tensor(0.0531)
tensor(0.0091)
tensor(0.0382)


So, what is there to do? Well, the first strategy people employ is to average the gradient across multiple individuals before uploading it to the central server. This strategy, however, will require some more sophisticated use of PointerTensor objects.